<a href="https://colab.research.google.com/github/Skuerjuan/tp2-chona/blob/main/maierowicz_atanasoff_gallo_ABD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2 Seminario IA
# Maierowicz, Atanasoff, Gallo
---
## ¿Deberían aceptar darle el préstamo?
### (Esta vez con redes neuronales)

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [52]:
# Preprocesamiento

data = "https://raw.githubusercontent.com/Skuerjuan/tp1_chona/refs/heads/main/loan_data.csv"
df = pd.read_csv(data)

categorical = ["person_gender", "person_education", "person_home_ownership", "loan_intent", "previous_loan_defaults_on_file"]

ct = ColumnTransformer(transformers=[("cat", OneHotEncoder(), categorical)], remainder="passthrough")

x = ct.fit_transform(df)

feature_names = ct.get_feature_names_out()

df = pd.DataFrame(x, columns=feature_names)
df_x = df.drop(columns="remainder__loan_status")
df_y = df["remainder__loan_status"]

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [53]:
# deberia haber un hyperparam de cuantas layers hay en el medio y otro de cuanto es el n
# otro más que sea el activation sería una buena idea (?)

model = keras.Sequential([
    layers.Input(shape=(x_train.shape[1],)),
    layers.Dense(15, activation="relu"),
    layers.Dense(2, activation="softmax")
])

model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 15)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 2)              │            32 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 452 (1.77 KB)

 Trainable params: 452 (1.77 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6624 - loss: 2740.8931
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7324 - loss: 38.5007
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7531 - loss: 22.5213
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7482 - loss: 29.5512
Epoch 5/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7546 - loss: 25.3243
Epoch 6/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7555 - loss: 27.0201
Epoch 7/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7741 - loss: 19.0763
Epoch 8/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7777 - loss: 19.5031
Epoch 9/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7725 - loss: 23.8618
Epoch 10/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7807 - loss: 21.4120


In [56]:
score = model.evaluate(x_test, y_test, batch_size=64, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

y_pred_probs = model.predict(x_test)
y_pred_labels = np.argmax(y_pred_probs, axis=1)

print(f'Precision: {precision_score(y_test, y_pred_labels)}\n')
print(f'Recall: {recall_score(y_test, y_pred_labels)}\n')

Test loss: 34.427425384521484
Test accuracy: 0.7846666574478149
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Precision: 1.0

Recall: 0.03582089552238806



In [ ]:
# evaluar el mejor set de hyperparams que quede al final
# comparar con el tp anterior